In [92]:
try:
    import mesa
except ImportError:
    !pip install mesa
    import mesa

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Resource Classes

In [93]:
class Sugar(mesa.Agent):

    def __init__(self, model, pos, max_sugar):
        super().__init__(model)
        #self.grid.place_agent(sugar, pos)
        #self.pos=pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):

        self.amount = min(self.amount + 1, self.max_sugar)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_sugar)



In [94]:
class Spice(mesa.Agent):

    def __init__(self, model, pos, max_spice):
        super().__init__(model)
        self.amount = max_spice
        self.max_spice = max_spice


    def step(self):

        self.amount = min(self.amount + 1, self.max_spice)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_spice)


In [98]:
class Trader(mesa.Agent):
    def __init__(self, model, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=0):

        super().__init__(model)
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision

    def is_occupied_by_other(self, pos):
        """
        helper function to check if the position is occupied by other traders for self.move()
        :return bool
        """
        if pos == self.pos:
            return False



    def move(self):

        # idenfiy all possible moves
        neighbors = [i for i in self.model.grid.get_neighborhood(self.pos, self.moore, include_center=True, radius=self.vision)
                     if not self.is_occupied_by_other(i)]




# model class

In [96]:
class SugarscapeG1mt(mesa.Model):
    def __init__(self, width=50, height=50, initial_population=200,
                 endowment_min=20, endowment_max=50,
                 metabolism_min=1, metabolism_max=5,
                 vision_min=1, vision_max=5):
        
        # 부모 클래스(mesa.Model) 초기화
        super().__init__()


        self.width = width
        self.height = height

        # Initialize
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max

        # .Grid property
        # torus=True로 하면 팩맨처럼 맵이 연결됩니다.
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

        self.sugar_distribution = np.genfromtxt("sugar-map.txt")
        self.spice_distribution = np.flip(self.sugar_distribution, axis=0)

        # 헬퍼 함수 make_map 호출
        self.make_map()


        for _, (x, y) in self.grid.coord_iter():
            max_sugar = self.sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(self, (x,y), max_sugar)
                self.grid.place_agent(sugar, (x, y))
                # self.schedule.add(sugar) # Mesa 3.0+에서는 필요 없으므로 삭제

            max_spice = self.spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(self, (x,y), max_spice)
                self.grid.place_agent(spice, (x, y))
                # self.schedule.add(spice) # Mesa 3.0+에서는 필요 없으므로 삭제

        for i in range(self.initial_population):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)

            sugar = self.random.randrange(self.endowment_min, self.endowment_max + 1)
            spice = self.random.randrange(self.endowment_min, self.endowment_max + 1)

            metabolism_sugar = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)
            metabolism_spice = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)

            vision = self.random.randrange(self.vision_min, self.vision_max + 1)

            # Initiate individual agent
            trader = Trader(self, (x,y), moore=False, # moore는 키워드 인자로 유지
                        sugar=sugar, spice=spice, # 나머지 인자들도 키워드 인자로 명시
                        metabolism_sugar=metabolism_sugar, metabolism_spice=metabolism_spice,
                        vision=vision)
            # Place agent
            self.grid.place_agent(trader, (x,y))
            # self.schedule.add(trader) # Mesa 3.0+에서는 필요 없으므로 삭제
            # print(trader.unique_id, trader.pos, trader.sugar, trader.spice, trader.metabolism_sugar,trader.metabolism_spice, trader.vision)

    def make_map(self):

        # 도화지를 가로로 길게(12x6 크기) 준비합니다.
        plt.figure(figsize=(12, 6))

        # 첫 번째 그림: 설탕 지도 (1행 2열 중 1번째)
        plt.subplot(1, 2, 1)
        plt.imshow(self.sugar_distribution, origin="lower", cmap="Oranges")
        plt.title("Sugar Map")
        plt.colorbar()

        # 두 번째 그림: 향신료 지도 (1행 2열 중 2번째)
        plt.subplot(1, 2, 2)
        plt.imshow(self.spice_distribution, origin="lower", cmap="BuPu")
        plt.title("Spice Map")
        plt.colorbar()


    def step(self):
        # 모델의 모든 에이전트(Sugar, Spice, Trader)의 step 메서드를 호출합니다.
        # Mesa 3.0+에서는 self.agents를 통해 에이전트들을 관리합니다.
        self.agents.do("step")

    #    for sugar in self.schedule.agents_by_type[Sugar].values():
    #        sugar.step()
        # for trader
        # agent death and removal
        # Mesa 3.0+에서는 self.agents.select()를 사용하여 특정 타입의 에이전트를 가져옵니다.
        trader_shuffle = list(self.agents.select(agent_type=Trader))
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()


    def run_model(self,step_count=1000):
        for i in range(step_count):
            self.step()


# Run Model


In [99]:
model = SugarscapeG1mt()
model.run_model(5)


I am 4301, at (8, 39), sugar: 40, spice: 48
I am 4254, at (48, 5), sugar: 20, spice: 41
I am 4271, at (47, 14), sugar: 46, spice: 26
I am 4313, at (0, 29), sugar: 46, spice: 33
I am 4258, at (44, 31), sugar: 36, spice: 20
I am 4142, at (33, 11), sugar: 33, spice: 23
I am 4177, at (47, 31), sugar: 49, spice: 31
I am 4230, at (27, 46), sugar: 30, spice: 21
I am 4250, at (32, 43), sugar: 36, spice: 34
I am 4251, at (17, 4), sugar: 32, spice: 45
I am 4246, at (43, 2), sugar: 37, spice: 50
I am 4202, at (23, 44), sugar: 34, spice: 47
I am 4262, at (28, 38), sugar: 26, spice: 37
I am 4193, at (12, 22), sugar: 26, spice: 44
I am 4317, at (44, 21), sugar: 33, spice: 24
I am 4155, at (18, 23), sugar: 37, spice: 27
I am 4266, at (3, 23), sugar: 49, spice: 50
I am 4184, at (33, 32), sugar: 25, spice: 22
I am 4257, at (38, 37), sugar: 24, spice: 30
I am 4265, at (42, 21), sugar: 32, spice: 50
I am 4306, at (32, 23), sugar: 43, spice: 32
I am 4163, at (49, 38), sugar: 41, spice: 21
I am 4316, at (8